In [10]:
#Load the library
library(Matrix)
library(dplyr)
library(Seurat)

In [7]:
#Load the expression matrix
dat_count <- readMM("10x_mouse_retina_development.mtx")
#Load the each coordinate of cells in three dimensional UMAP space
dat_phenotype <- read.csv("10x_mouse_retina_development_phenotype.csv", header=T, row.names=1, stringsAsFactors=F)
#Load the annotation file
dat_feature <- read.csv("10x_mouse_retina_development_feature.csv", header=T, stringsAsFactors=F)

In [8]:
#Check the data dimensions
dim(dat_count)
dim(dat_phenotype)
dim(dat_feature)

[1]  27998 107052

[1] 107052    100

[1] 27998     8

In [20]:
#Show the subset of the data
head(dat_count)

6 x 107052 sparse Matrix of class "dgTMatrix"
                                                                               
[1,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
[2,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
[3,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
[4,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
[5,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
[6,] . . . . . . . . . . . . . . . . . . . . . . 1 . . . . . . . . . . . ......

 .....suppressing columns in show(); maybe adjust 'options(max.print= *, width = *)'
 ..............................

In [21]:
#Show the subset of the data
head(dat_phenotype)

,barcode,sample,age,Size_Factor,num_genes_expressed,Total_mRNAs,CellType,raw_cluster,new_CellType,largeVis1,⋯,Patt.71,Patt.72,Patt.73,Patt.74,Patt.75,Patt.76,Patt.77,Patt.78,Patt.79,Patt.80
,<chr>,<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
E11.AAACCTGAGATGTAAC.1,E11.AAACCTGAGATGTAAC-1,E11,E11,1.733369,1549,2794,RPCs,32,Early RPCs,2.2338175,⋯,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
E11.AAACCTGAGGCAATTA.1,E11.AAACCTGAGGCAATTA-1,E11,E11,2.201143,1848,3548,RPCs,9,Early RPCs,-0.6808101,⋯,1.008011,21.921519,16.938427,1.000011,1.000180,60.656578,1.002151,37.660817,1.000047,1.000198
E11.AAACCTGAGTAGCGGT.1,E11.AAACCTGAGTAGCGGT-1,E11,E11,2.381057,1935,3838,RPCs,24,RPE/Margin/Periocular Mesenchyme/Lens Epithelial Cells,1.6031758,⋯,250.307592,1.004293,1.000764,1.002577,1.000022,1.000262,1.000022,1.170338,1.001827,1.000559
E11.AAACCTGAGTGGAGTC.1,E11.AAACCTGAGTGGAGTC-1,E11,E11,2.861238,2139,4612,RPCs,9,Early RPCs,-0.7359965,⋯,1.016430,50.007646,1.011516,1.002841,1.000161,41.925894,1.001989,8.626739,1.021516,1.001392
E11.AAACCTGCAAGTTGTC.1,E11.AAACCTGCAAGTTGTC-1,E11,E11,2.044805,1723,3296,Lens Epithelia,74,RPE/Margin/Periocular Mesenchyme/Lens Epithelial Cells,2.7458238,⋯,1.000668,19.482778,1.000139,1.018392,1.000059,1.000044,1.010160,1.000184,1.000493,3.845403
E11.AAACCTGCACCCATTC.1,E11.AAACCTGCACCCATTC-1,E11,E11,2.280553,1806,3676,RPCs,32,Early RPCs,1.9579134,⋯,1.000071,1.211624,1.006995,1.000126,1.000037,1.001017,1.003272,248.890062,1.000286,1.000414


In [22]:
#Show the subset of the data
head(dat_feature)

X,id,gene_short_name,num_cells_expressed,mean_expr,sd_exprs,bcv,percent_detection
<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
ENSMUSG00000051951,ENSMUSG00000051951,Xkr4,253,2.119135e-03,0.046522355,481.95455,0.209430151
ENSMUSG00000089699,ENSMUSG00000089699,Gm1992,4,3.311149e-05,0.005754187,30200.24999,0.003311149
ENSMUSG00000102343,ENSMUSG00000102343,Gm37381,58,4.801166e-04,0.021906392,2081.84482,0.048011655
ENSMUSG00000025900,ENSMUSG00000025900,Rp1,11816,1.694811e-01,0.635347014,14.05333,9.781133075
ENSMUSG00000109048,ENSMUSG00000109048,Rp1,0,0.000000e+00,0.000000000,NA,0.000000000
ENSMUSG00000025902,ENSMUSG00000025902,Sox17,108,1.622463e-03,0.077612551,2288.30757,0.089401013


Below procedure requires a lots of memory, so I partially performed analysis in a HPC cluster

In [ ]:
#Creat Seurat class object
colnames(dat_count) <- rownames(dat_phenotype)
rownames(dat_count) <- rownames(dat_feature)
dat <- CreateSeuratObject(counts = dat_count, project = "Clark_P2_Retina", meta.data = dat_phenotype)

In [30]:
#Leave only cells in P2
dat <- subset(dat, subset = (age=="P2"))

[1] "dat_count"     "dat_feature"   "dat_phenotype" "data_feature"

In [ ]:
#Save dat object in RDS format
 saveRDS(dat, file="Clark_P2_Retina.rds")

From here, we go back to local laptop environment

In [2]:
#Load the library
library(Matrix)
library(Seurat)

#Load dat object
dat <- readRDS(file = "Clark_P2_Retina.rds")

In [3]:
#Structure of dat object
str(dat)

Formal class 'Seurat' [package "Seurat"] with 12 slots
  ..@ assays      :List of 1
  .. ..$ RNA:Formal class 'Assay' [package "Seurat"] with 7 slots
  .. .. .. ..@ counts       :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:18425089] 16 26 62 63 70 150 152 179 181 212 ...
  .. .. .. .. .. ..@ p       : int [1:15583] 0 1144 2376 3256 4695 6018 7240 8903 10089 11395 ...
  .. .. .. .. .. ..@ Dim     : int [1:2] 27998 15582
  .. .. .. .. .. ..@ Dimnames:List of 2
  .. .. .. .. .. .. ..$ : chr [1:27998] "1" "2" "3" "4" ...
  .. .. .. .. .. .. ..$ : chr [1:15582] "P2_rep2.AAACCTGAGATAGGAG.1" "P2_rep2.AAACCTGAGCATCATC.1" "P2_rep2.AAACCTGAGCCTATGT.1" "P2_rep2.AAACCTGAGCTAGCCC.1" ...
  .. .. .. .. .. ..@ x       : num [1:18425089] 1 3 5 1 1 1 1 1 1 1 ...
  .. .. .. .. .. ..@ factors : list()
  .. .. .. ..@ data         :Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  .. .. .. .. .. ..@ i       : int [1:18425089] 16 26 62 63 70 150 152